<a href="https://colab.research.google.com/github/rhiats/TIHM-Dataset-RNN/blob/main/RNN_TIHM_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from torch.nn.utils.rnn import pad_sequence

**Create Sequences**

In [3]:
df = pd.read_csv("/content/drive/MyDrive/TIHM_Dataset/Dataset/Sleep.csv")

In [4]:
df.head()

,patient_id,date,state,heart_rate,respiratory_rate,snoring
0,0f352,2019-06-25 22:53:00,AWAKE,69.0,14.0,False
1,0f352,2019-06-25 22:54:00,AWAKE,66.0,14.0,False
2,0f352,2019-06-25 22:55:00,AWAKE,70.0,14.0,False
3,0f352,2019-06-25 22:56:00,AWAKE,70.0,13.0,False
4,0f352,2019-06-25 22:57:00,AWAKE,68.0,13.0,False


In [5]:
df.sort_values(['patient_id', 'date'], ascending=[True, True])

,patient_id,date,state,heart_rate,respiratory_rate,snoring
0,0f352,2019-06-25 22:53:00,AWAKE,69.0,14.0,False
1,0f352,2019-06-25 22:54:00,AWAKE,66.0,14.0,False
2,0f352,2019-06-25 22:55:00,AWAKE,70.0,14.0,False
3,0f352,2019-06-25 22:56:00,AWAKE,70.0,13.0,False
4,0f352,2019-06-25 22:57:00,AWAKE,68.0,13.0,False
...,...,...,...,...,...,...
461418,f220c,2019-06-30 10:47:00,AWAKE,76.0,20.0,False
461419,f220c,2019-06-30 10:48:00,AWAKE,73.0,21.0,False
461420,f220c,2019-06-30 10:49:00,AWAKE,65.0,18.0,False
461421,f220c,2019-06-30 10:50:00,AWAKE,75.0,15.0,False


In [8]:
features = ['heart_rate', 'respiratory_rate']
target = ['snoring']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

df.head()

,patient_id,date,state,heart_rate,respiratory_rate,snoring
0,0f352,2019-06-25 22:53:00,AWAKE,0.449275,0.260870,False
1,0f352,2019-06-25 22:54:00,AWAKE,0.405797,0.260870,False
2,0f352,2019-06-25 22:55:00,AWAKE,0.463768,0.260870,False
3,0f352,2019-06-25 22:56:00,AWAKE,0.463768,0.217391,False
4,0f352,2019-06-25 22:57:00,AWAKE,0.434783,0.217391,False


In [11]:
print(len(df[features].values))

461423


In [27]:
def create_sequences(data, target, seq_length=10):
    sequences = []
    labels = []

    #Create sequences of length 10
    for i in range(len(data) - seq_length):
      sequences.append(data[i:i+seq_length])
      labels.append(target[i+seq_length])
    return np.array(sequences), np.array(labels)

In [28]:
X, y = create_sequences(df[features].values, df[target].values)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

**RNN Model**

In [34]:
# Define RNN Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # Initial hidden state
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Output from last time step
        return out

**Model parameters**

In [35]:
input_size = len(features)
hidden_size = 32
output_size = 1  # Regression or Binary classification

**Model**

In [37]:
model = RNN(input_size, hidden_size, output_size)

**Loss using Binary Classification**

In [38]:
criterion = nn.MSELoss()  # Use BCEWithLogitsLoss() for binary classification